In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('../data/AviationData.csv', encoding='latin-1')

df['Event.Date'] = pd.to_datetime(df['Event.Date'])
#Creating a column for the year
df['Year'] = df['Event.Date'].dt.year
#Making everything in the df lowercase so we can get more accurate counts.
df = df.applymap(lambda x: x.lower() if type(x) == str else x)
# Filtering out 'Amateur.Built' aircraft
df = df[df['Amateur.Built'] == 'no']
# And only keep 'airplane' from the 'Aircraft.Category' column
df = df[df['Aircraft.Category'] == 'airplane']
df.info()

c:\Users\nateh\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24417 entries, 5 to 88886
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                24417 non-null  object        
 1   Investigation.Type      24417 non-null  object        
 2   Accident.Number         24417 non-null  object        
 3   Event.Date              24417 non-null  datetime64[ns]
 4   Location                24411 non-null  object        
 5   Country                 24410 non-null  object        
 6   Latitude                19171 non-null  object        
 7   Longitude               19165 non-null  object        
 8   Airport.Code            15574 non-null  object        
 9   Airport.Name            16012 non-null  object        
 10  Injury.Severity         23604 non-null  object        
 11  Aircraft.damage         23147 non-null  object        
 12  Aircraft.Category       24417 non-null  object

In [9]:
# Cleaning up the FAR.Description column using a .replace() 
# See here for more info on FAR: 
# https://pilotinstitute.com/part-91-vs-121-vs-135/

df['FAR.Description'] = df['FAR.Description'].replace({
    '091': 'part 91',
    'part 91: general aviation': 'part 91',
    '137': 'part 137',
    '135': 'part 135',
    '121': 'part 121',
    'part 137: agricultural': 'part 137',
    'part 135: air taxi & commuter': 'part 135',
    '129': 'part 129',
    'part 121: air carrier': 'part 121',
    '133': 'part 133',
    'part 129: foreign': 'part 129',
    'part 133: rotorcraft external load': 'part 133',
    'part 133: rotorcraft ext. load': 'part 133',
    '125' : 'part 125',
    'part 125: 20+ pax,6000+ lbs': 'part 125',
    '107': 'part 107',
    '103': 'part 103',
    '437': 'part 437',
    'part91f': 'part 91',
    'part 91 subpart f': 'part 91',
    'part 91f: special flt ops.': 'part 91',
    'part 91 subpart k: fractional': 'part 91',
    '091k': 'part 91',
    'pubu': 'public',
    'public use': 'public',
    'public aircraft': 'public',
    'armed forces': 'armf',
}
)

df['FAR.Description'].value_counts()

part 91                     18502
nusn                         1122
part 137                     1102
part 121                      814
nusc                          804
part 135                      773
part 129                      280
unk                           228
public                        138
non-u.s., non-commercial       72
non-u.s., commercial           67
unknown                        18
part 125                       10
armf                            4
part 133                        2
part 107                        1
Name: FAR.Description, dtype: int64

In [10]:
#Now grouping the 'Purpose.of.flight' category into more general categories

df['purpose_binned'] = df['Purpose.of.flight'].replace(['public aircraft', 'public aircraft - federal', 'public aircraft - state', 'public aircraft - local', 'public aircraft - u.s. govt.'], 'public')
# Grouping 'aerial application', 'business', 'other work use', 'banner tow', 'skydiving', 'glider tow', 'air drop' into 'business'
df['purpose_binned'] = df['purpose_binned'].replace(['aerial application', 'business', 'other work use', 'banner tow', 'skydiving', 'glider tow', 'air drop'], 'business')
# Grouping 'executive/corporate', 'ferry' into 'small transport'.
df['purpose_binned'] = df['purpose_binned'].replace(['executive/corporate', 'ferry'], 'small transport')
# Adding more into the public category
df['purpose_binned'] = df['purpose_binned'].replace(['pubs', 'public', 'publ', 'firefighting'], 'public')
# Adding 'flight test' to 'instructional
df['purpose_binned'] = df['purpose_binned'].replace(['flight test'], 'instructional')
# Grouping air show categories
df['purpose_binned'] = df['purpose_binned'].replace(['air race show', 'air race/show', 'asho', 'air show'])

df['purpose_binned'].value_counts()

personal              13377
instructional          3245
business               2283
unknown                 839
positioning             350
small transport         311
aerial observation      171
public                  150
external load             1
Name: purpose_binned, dtype: int64